**Importing necessary functions**

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import numpy as np 
import pylab 
import scipy.stats as stats 
from scipy.stats import boxcox
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Lasso
from  sklearn.linear_model import Ridge
from  sklearn.preprocessing  import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.svm import SVR

**# Importing the data file and renaming the columns** 


In [ ]:
#reading the file
df=pd.read_csv('Twitter.data')

#renaming the columns
data_cols=['NCD_0','NCD_1','NCD_2','NCD_3','NCD_4','NCD_5','NCD_6','AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6','AS(NA)_0','AS(NA)_1','AS(NA)_2','AS(NA)_3','AS(NA)_4','AS(NA)_5','AS(NA)_6','BL_0','BL_1','BL_2','BL_3','BL_4','BL_5','BL_6','NAC_0','NAC_1','NAC_2','NAC_3','NAC_4','NAC_5','NAC_6','AS(NAC)_0','AS(NAC)_1','AS(NAC)_2','AS(NAC)_3','AS(NAC)_4','AS(NAC)_5','AS(NAC)_6','CS_0','CS_1','CS_2','CS_3','CS_4','CS_5','CS_6','AT_0','AT_1','AT_2','AT_3','AT_4','AT_5','AT_6','NA_0','NA_1','NA_2','NA_3','NA_4','NA_5','NA_6',
'ADL_0','ADL_1','ADL_2','ADL_3','ADL_4','ADL_5','ADL_6','NAD_0','NAD_1','NAD_2','NAD_3','NAD_4','NAD_5','NAD_6','Target']
df.columns=data_cols
df.shape

In [ ]:
df.describe()

**Data Leakage , the correlation between target and features are evaluated and Highly correlated values are chosen .


In [ ]:
#selecting the required columns
columns = ['NCD_0','NCD_1','NCD_2','NCD_3','NCD_4','NCD_5','NCD_6','AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6','AS(NA)_0','AS(NA)_1','AS(NA)_2','AS(NA)_3','AS(NA)_4','AS(NA)_5','AS(NA)_6','BL_0','BL_1','BL_2','BL_3','BL_4','BL_5','BL_6','NAC_0','NAC_1','NAC_2','NAC_3','NAC_4','NAC_5','NAC_6','AS(NAC)_0','AS(NAC)_1','AS(NAC)_2','AS(NAC)_3','AS(NAC)_4','AS(NAC)_5','AS(NAC)_6','CS_0','CS_1','CS_2','CS_3','CS_4','CS_5','CS_6','AT_0','AT_1','AT_2','AT_3','AT_4','AT_5','AT_6','NA_0','NA_1','NA_2','NA_3','NA_4','NA_5','NA_6',
'ADL_0','ADL_1','ADL_2','ADL_3','ADL_4','ADL_5','ADL_6','NAD_0','NAD_1','NAD_2','NAD_3','NAD_4','NAD_5','NAD_6']

for name in columns:
    print(name,np.corrcoef(df[name],df['Target'])[0,1])

**The following features and hence chose to proceed further . 
But the other columns are ignored . **  'AC_1','NAC_2','NAC_3','NAC_4','NAC_5','NAC_6','AS(NAC)_0','AS(NAC)_1','AS(NAC)_2','AS(NAC)_3','AS(NAC)_4','AS(NAC)_5','AS(NAC)_6','CS_0','CS_1','CS_2','CS_3','CS_4','CS_5','CS_6','NA_0','NA_1','NA_2','NA_3','NA_4','NA_5','NA_6','NAD_0','NAD_1','NAD_2','NAD_3','NAD_4','NAD_5','NAD_6'

In [ ]:
['AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6','AS(NA)_0','AS(NA)_1','AS(NA)_2','AS(NA)_3','AS(NA)_4','AS(NA)_5','AS(NA)_6','NAC_0','NAC_1','NAC_2','NAC_3','NAC_4','NAC_5','NAC_6','AS(NAC)_0','AS(NAC)_1','AS(NAC)_2','AS(NAC)_3','AS(NAC)_4','AS(NAC)_5','AS(NAC)_6','CS_0','CS_1','CS_2','CS_3','CS_4','CS_5','CS_6','NA_0','NA_1','NA_2','NA_3','NA_4','NA_5','NA_6','NAD_0','NAD_1','NAD_2','NAD_3','NAD_4','NAD_5','NAD_6']

**To remove the multicollinearity between variables - we are calculating the VIF for each variable - threshold < 5 **

In [ ]:
#Creating a data frame with just the variables which are correlated to Target
#extracting the features as X and target as Y 
X = df[['AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6','AS(NA)_0','AS(NA)_1','AS(NA)_2','AS(NA)_3','AS(NA)_4','AS(NA)_5','AS(NA)_6','NAC_0','NAC_1','NAC_2','NAC_3','NAC_4','NAC_5','NAC_6','AS(NAC)_0','AS(NAC)_1','AS(NAC)_2','AS(NAC)_3','AS(NAC)_4','AS(NAC)_5','AS(NAC)_6','CS_0','CS_1','CS_2','CS_3','CS_4','CS_5','CS_6','NA_0','NA_1','NA_2','NA_3','NA_4','NA_5','NA_6','NAD_0','NAD_1','NAD_2','NAD_3','NAD_4','NAD_5','NAD_6','Target']]
y = X.pop('Target')

In [ ]:
X.head()

In [ ]:
#Creating a class which can calculate VIF for each variable
class ReduceVIF(BaseEstimator, TransformerMixin):
    def __init__(self, thresh=5.0, impute=True, impute_strategy='median'):
        # From looking at documentation, values between 5 and 10 are "okay".
        # Above 10 is too high and so should be removed.
        self.thresh = thresh
        
        # The statsmodel function will fail with NaN values, as such we have to impute them.
        # By default we impute using the median value.
        # This imputation could be taken out and added as part of an sklearn Pipeline.
        if impute:
            self.imputer = Imputer(strategy=impute_strategy)

    def fit(self, X, y=None):
        print('ReduceVIF fit')
        if hasattr(self, 'imputer'):
            self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        if hasattr(self, 'imputer'):
            X = pd.DataFrame(self.imputer.transform(X), columns=columns)
        return ReduceVIF.calculate_vif(X, self.thresh)

    @staticmethod
    def calculate_vif(X, thresh=5.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            variables = X.columns
            dropped = False
            vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
        return X

In [ ]:
transformer = ReduceVIF()

In [ ]:
#Analyzing which variables to drop - to remove multicollinearity
X = transformer.fit_transform(X[X.columns[-49:]], y)

Finally decided to remove variables which are more than VIF 10 - Based on commonly used assumption and the below columns are chosen for further steps. 

To avoid overfitting and train regression models better with data values better  these steps has been carried out. ['AI_0','AI_1','AI_2','AI_4','AI_5','AI_6','AS(NA)_3','AS(NAC)_0','AS(NAC)_2','CS_0','CS_1','NAD_3']

In [ ]:
['AI_0','AI_1','AI_2','AI_4','AI_5','AI_6','AS(NA)_3','AS(NAC)_0','AS(NAC)_2','CS_0','CS_1','NAD_3']

**Drawing distributions for each feature of AI**

In [ ]:
data_cols=['AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6']

for name in data_cols:
    print(name)
    x = df[name]
    plt.hist(x, bins=100)
    plt.ylabel('No of times')
    plt.show()
    plt.tight_layout()

**Since the columns is not normally distrubuted, we are trying various combinations of log,sqrt and box cox to reduce skewness **

In [ ]:
#understanding the skewness of AI_0
from scipy.stats import boxcox
from scipy.stats import skew
import math

skness = skew(df['AI_0'])
print("skness",skness)

x11=np.sqrt(df['AI_0'])
skness_sqrt = skew(x11)
print("skness_sqrt",skness_sqrt)

x12 = (x11+1).apply(np.log)
skness_sqrt_log = skew(x12)
print("skness_sqrt_log",skness_sqrt_log) 

bxcx = boxcox(x12+1)[0] #---- we are selecting this transformation
skness_sqrt_log_bxcx = skew(bxcx)
print("skness_sqrt_log_bxcx",skness_sqrt_log_bxcx)

x = (df['AI_0']+1).apply(np.log)
skness_log = skew(x11)
print("skness_log",skness_log)

x11=np.sqrt(x)
skness_log_sqrt = skew(x11)
print("skness_log_sqrt",skness_log_sqrt)

bxcx = boxcox(x11+1)[0]
skness_log_sqrt_bxcx = skew(bxcx)
print("skness_log_sqrt_bxcx",skness_log_sqrt_bxcx)

**Now applying the same transformation techniques(sqrt, log and box cox) for other AI columns i.e. from AI_1 to AI_6 **

In [ ]:
data_cols_AI=['AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6']

for name in data_cols_AI:
    print(name)
    from scipy.stats import boxcox
    from scipy.stats import skew
    import math

    skness = skew(df[name])
    print("skness",skness)

    x11=np.sqrt(df[name])
    skness_sqrt = skew(x11)
    print("skness_sqrt",skness_sqrt)

    x12 = (x11+1).apply(np.log)
    skness_sqrt_log = skew(x12)
    print("skness_sqrt_log",skness_sqrt_log) #---- we are selecting this transformation
    
    bxcx = boxcox(x12+1)[0] #---- we are selecting this transformation
    skness_sqrt_log_bxcx = skew(bxcx)
    print("skness_sqrt_log_bxcx",skness_sqrt_log_bxcx)   

    #printing the q-q plot and histogram for the selected transformation - sqrt_log
    #Checking the normality by drawing a Q-Q plot on transformed data 
    print("Q-Q Plot for the transformed column")
    stats.probplot(bxcx, dist="norm", plot=pylab)
    pylab.show()

    #Checking the histogram of the untransformed data 
    print("histogram for the transformed data")
    plt.hist(bxcx, bins='auto')
    plt.ylabel('No of times')
    plt.show()

**Drawing distributions for each variable of  AS(NA)_3 ,AS(NAC)_0 ,AS(NAC)_2 ,CS_0 ,CS_1 ,NAD_3 **

In [ ]:
from scipy.stats import boxcox
from scipy.stats import skew
import math
data_cols=['AS(NA)_3','AS(NAC)_0','AS(NAC)_2','CS_0','CS_1','NAD_3']
#data_cols_AI=['AI_0','AI_1','AI_2','AI_3','AI_4','AI_5','AI_6']

for name in data_cols:
    print(name)
    from scipy.stats import boxcox
    from scipy.stats import skew
    import math

    skness = skew(df[name])
    print("skness",skness)

    x11=np.sqrt(df[name])
    skness_sqrt = skew(x11)
    print("skness_sqrt",skness_sqrt)

    x12 = (x11+1).apply(np.log)
    skness_sqrt_log = skew(x12)
    print("skness_sqrt_log",skness_sqrt_log) #---- we are selecting this transformation
    
    bxcx = boxcox(x12+1)[0] #---- we are selecting this transformation
    skness_sqrt_log_bxcx = skew(bxcx)
    print("skness_sqrt_log_bxcx",skness_sqrt_log_bxcx)   

    #printing the q-q plot and histogram for the selected transformation - sqrt_log
    #Checking the normality by drawing a Q-Q plot on transformed data 
    print("Q-Q Plot for the transformed column")
    stats.probplot(bxcx, dist="norm", plot=pylab)
    pylab.show()

    #Checking the histogram of the untransformed data 
    print("histogram for the transformed data")
    plt.hist(bxcx, bins='auto')
    plt.ylabel('No of times')
    plt.show()


After determining the type of transformation  , lets apply that transformed value in the data frame 

The sqrt-log transformation is applied to features : NCD_1,NCD_2,NCD_3,'AI_1','AI_2','AI_3'

The square root , log transformation and boxcox transformations is applied to the following features:    NCD_0,NCD_4,NCD_5,NCD_6,'AI_0','AI_4','AI_5','AI_6'

In [ ]:
sqrt_log_bxcx=['AI_0','AI_1','AI_5','AI_6','AS(NA)_3','AS(NAC)_0','AS(NAC)_2']
sqrt_log =['AI_2','AI_3','AI_4','NAD_3']
td=df
for names1 in sqrt_log_bxcx: 
   #td[names1]=boxcox(np.sqrt(df[names1]).apply(np.log)+1)[0] 
    x=np.sqrt(df[names1])
    x11 = (x+1).apply(np.log)
    x12 = boxcox(x11+1)[0]
    td[names1] = x12
    
for names2 in sqrt_log: 
    x2=np.sqrt(df[names2])
    x21 = (x2+1).apply(np.log)
    td[names2] = x21  

In [ ]:
x_train=td[['AI_2','AI_3','AI_4','NAD_3','AI_0','AI_1','AI_5','AI_6','AS(NA)_3','AS(NAC)_0','AS(NAC)_2']]
y=td['Target']


Performing the testing Train split for all the data values for the transformed columns  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train,y, random_state = 0,test_size=0.2)


After preforming scaling of data and running the models , it was observed that there was not much of impact in results due to scaling and the code has been commented. 

In [ ]:
#from sklearn.preprocessing import MinMaxScaler 

#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train_unscaled)
#X_test = scaler.transform(X_text_unscaled)

##Linear Regression

In [ ]:

lreg = LinearRegression()
a=lreg.fit(X_train, y_train)
print('training_score',lreg.score(X_train, y_train))
print('test_score',lreg.score(X_test, y_test))
#a.summary()


#Polynomial  Regression

In [ ]:
train_score_list = []
test_score_list = []

for n in range(1,4):
    poly = PolynomialFeatures(n)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    lreg.fit(X_train_poly, y_train)
    train_score_list.append(lreg.score(X_train_poly, y_train))
    test_score_list.append(lreg.score(X_test_poly, y_test))
print(train_score_list)
print(test_score_list)

In [ ]:
%matplotlib inline

x_axis = range(1,4)
plt.plot(x_axis, train_score_list, c = 'g', label = 'Train Score')
plt.plot(x_axis, test_score_list, c = 'b', label = 'Test Score')
plt.xlabel('degree')
plt.ylabel('accuracy')
plt.legend()

In [ ]:
#trial for alpha values for polynomial ridge
x_range = [0.01, 0.1, 1, 10, 100]

train_score_list = []
test_score_list = []

poly = PolynomialFeatures(3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
    
for alpha_1 in x_range: 
    ridge = Ridge(alpha_1)
    ridge.fit(X_train_poly,y_train)
    train_score_list.append(ridge.score(X_train_poly,y_train))
    test_score_list.append(ridge.score(X_test_poly, y_test))
       
    print(train_score_list)
    print(test_score_list)
    



In [ ]:
print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in [0, 1, 10]:
    poly = PolynomialFeatures(3)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    linridge = Ridge(alpha = this_alpha).fit(X_train_poly, y_train)
    r2_train = linridge.score(X_train_poly, y_train)
    r2_test = linridge.score(X_test_poly, y_test)
    print("train_score  poly ridge",  r2_train)
    print("test_score  poly ridge",r2_test)
    
    

In [ ]:
# finding best alpha by gridsearch
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': [ 0.001,0.01,0.1,0,1]}
grid_search = GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_train_poly, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

grid_search= GridSearchCV(ridge(),param_grid, cv=5)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
##Polynomial ridge degree 2 with alpha 0.001 as best param 

poly = PolynomialFeatures(3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
linridge = Ridge(alpha =0).fit(X_train_poly, y_train)
r2_train = linridge.score(X_train_poly, y_train)
r2_test = linridge.score(X_test_poly, y_test)
print("train_score  poly ridge ",  r2_train)
print("test_score  poly ridge",r2_test)

In [ ]:
print('Lasso regression: effect of alpha regularization parameter\n')
for this_alpha in [0.1,0,1]:
    linlasso = Lasso(alpha = this_alpha).fit(X_train_poly, y_train)
    r2_train = linlasso.score(X_train_poly, y_train)
    r2_test = linlasso.score(X_test_poly, y_test)
    num_coeff_bigger = np.sum(abs(linlasso.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
    r-squared training: {:.2f}, r-squared test: {:.2f}\n'
    .format(this_alpha, num_coeff_bigger, r2_train, r2_test))

In [ ]:
x_pred

In [ ]:
%matplotlib inline
train_score_array = []
test_score_array = []

for k in range(1,10):
    knn_reg = KNeighborsRegressor(k)
    knn_reg.fit(X_train, y_train)
    train_score_array.append(knn_reg.score(X_train, y_train))
    test_score_array.append(knn_reg.score(X_test, y_test))

x_axis = range(1,10)
plt.plot(x_axis, train_score_array, c = 'g', label = 'Train Score')
plt.plot(x_axis, test_score_array, c = 'b', label = 'Test Score')
plt.legend()
plt.xlabel('k')
plt.ylabel('MSE')

#N neighbors with 2 value is best param in knn regression
knnreg = KNeighborsRegressor(n_neighbors = 2).fit(X_train, y_train)
print('R-squared test score: {:.3f}'.format(knnreg.score(X_test, y_test)))
print('R-squared train score: {:.3f}'.format(knnreg.score(X_train, y_train)))
#print('MAE for train data set :', metrics.mean_absolute_error(y_train, linridge.predict(X_train)))
#x_pred=knnreg.predict(X_test)



In [ ]:
%matplotlib inline

#x_axis = range(1,4)
plt.scatter(x_pred, y_test)
#plt.scatter(y_test, c = 'b')
plt.xlabel('Predicted values of number of active discussions' )
plt.ylabel('actual  Buzz number of active discussions')
plt.legend()

In [ ]:
# creating SVR model with C=1000 and gamma=1
svr = SVR( epsilon = 0.01,kernel='linear', C=1, gamma=100)
#svr_rbf = SVR( epsilon = 0.01,kernel='rbf', C=1000, gamma=0.001)

#calculating score and RME
svr.fit(X_train, y_train)

print('R-squared  score(train) : {:.3f}'
     .format(svr.score(X_train,y_train)))
print('R-squared  score(test) : {:.3f}'
     .format(svr.score(X_test,y_test)))
print('MAE for train data set :', metrics.mean_absolute_error(y_train, svr.predict(X_train)))
svr_rbf.fit(X_train_scaled, y_train)

print('R-squared  score(train) : {:.3f}'
     .format(svr_rbf.score(X_train_scaled,y_train)))
print('R-squared  score(test) : {:.3f}'
     .format(svr_rbf.score(X_test,y_test)))
print('MAE for train data set :', metrics.mean_absolute_error(y_train, svr_rbf.predict(X_train_scaled)))

The models where executed and the best model is polynomial ridge with R square value for training is 0.8424 and R score for testing score is 0.80 and The best parameter for alpha is 0 .